  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi , Qiuqi Luo and Beihui Hu


#*************************************************************************************************************

Copyright (c) 2020, Ankit Kariryaa

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.s

#*************************************************************************************************************


### Overview 

The code was written by Ankit Kariryaa (Kariryaa AT uni-bremen DOT de) in 2018 (see https://doi.org/10.5281/zenodo.3978185), and some modifications were made by Xuehui Pi and Qiuqi Luo in 2020, some modifications were made by Beihui Hu in 2024,.

Start by labeling a part of the satellite images with the lakes and storing the labels in shapefiles. The areas that are labeled are denoted by the 'training area' and actual lakes in that area are denoted by the 'training polygons'.

- First, we read the training area and the training polygons from two separate shapefiles. Then we determine the training area for each training polygon. 
- Next, we read the parameters of each training area,for each area, parameter 'id' means the id of raw satellite image, 'type' means the region type of this area, 'file name' means the suffix file name of saved images.
- Finally, we read and clip the raw satellite image (mean-NDWI,mean-B4,mean-B3,mean-B2,mea-B11) of each area, obtain the corresponding image files and label files, and store them into three subfolders ('train', 'test' and 'val') based on the data type.

In [ ]:
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
from rasterio import features
from shapely.geometry import box

import numpy as np      
import os
from core.visualize import display_images

%matplotlib inline
from tqdm import tqdm_notebook as tqdm
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/Preprocessing.py)
# Please provide required info in the file before continuing with this notebook. 
# hbh: in this scene,a new config named Preprocessing_within is created to distinguish from the original
from config import Preprocessing   
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import Preprocessing
config = Preprocessing.Configuration()

In [ ]:
#hbh: check whether the output dir(must be present) of each type is empty
for dataset in ['train','test','val']:
    dataset_dir=os.path.join(config.dataset_dir,'{}'.format(dataset))
    os.makedirs(dataset_dir,exist_ok=True)

In [ ]:
#Read the training area 、 training polygons
trainingArea = gps.read_file(os.path.join(config.training_base_dir, config.training_area_fn))
trainingPolygon = gps.read_file(os.path.join(config.training_base_dir, config.training_polygon_fn))

print(trainingPolygon.shape,trainingArea.shape)# area:id, geomerry;   polygon:id, geometry 
trainingPolygon
trainingArea
print(f'Read a total of {trainingPolygon.shape[0]} object polygons and {trainingArea.shape[0]} training areas.')
print(f'Polygons will be assigned to training areas in the next steps.')

In [ ]:
trainingPolygon=trainingPolygon[trainingPolygon['dataset']=='test']
trainingArea=trainingArea[trainingArea['dataset']=='test']
trainingArea=trainingArea[-1:]
trainingArea

In [ ]:
# Check if the training areas and the training polygons have the same crs     
if trainingArea.crs  != trainingPolygon.crs:
    print('Training area CRS does not match training_polygon CRS')
    targetCRS = trainingPolygon.crs #Areas are less in number so conversion should be faster
    trainingArea = trainingArea.to_crs(targetCRS)
print(trainingPolygon.crs)
print(trainingArea.crs)
assert trainingPolygon.crs == trainingArea.crs

In [ ]:
# As input we received two shapefile, first one contains the training areas/rectangles and other contains the polygon of lakes/objects in those training areas
# The first task is to determine the parent training area for each polygon.

def dividePolygonsInTrainingAreas(trainingPolygon, trainingArea):
    '''Assign annotated ploygons in to the training areas.'''
    # For efficiency, assigned polygons are removed from the list, we make a copy here. 
    cpTrainingPolygon = trainingPolygon.copy()
    splitPolygons = {}
    for i in tqdm(trainingArea.index):
        spTemp = [] 
        allocated = []
        print("area's index:",i)
        for j in cpTrainingPolygon.index:
            if cpTrainingPolygon.loc[j]['geometry'].intersects(trainingArea.loc[i]['geometry']):
                spTemp.append(cpTrainingPolygon.loc[j])
                allocated.append(j)      
        splitPolygons[i] = {'polygons':spTemp,'bounds':list(trainingArea.bounds.loc[i]),'type':trainingArea.loc[i]['type'],'dataset':trainingArea.loc[i]['dataset'],'grid_id':trainingArea.loc[i]['grid_id'] ,'file_name':trainingArea.loc[i]['file_name'] }
        cpTrainingPolygon = cpTrainingPolygon.drop(allocated)#assigned polygons are removed from the list
    return splitPolygons

# areasWithPolygons contains the object polygons for each area!
areasWithPolygons = dividePolygonsInTrainingAreas(trainingPolygon, trainingArea)
print(f'Assigned training polygons in {len(areasWithPolygons)} training areas')

In [ ]:
print(areasWithPolygons)

In [ ]:
def extractAreasThatOverlapWithTrainingData(areaInfo, writePath):
    """Iterates over raw NDWI images and using findOverlap() extract areas that overlap with training data. 
    The overlapping areas in raw images are written in a separate file, and annotation file are created from polygons in the overlapping areas.
    """
    if not os.path.exists(writePath):
        os.makedirs(writePath)
        
    polygonsInAreaDf = gps.GeoDataFrame(areaInfo['polygons'])
    grid_id=str(areaInfo['grid_id'])
    file_name=str(areaInfo['file_name'])
    print(grid_id)
    bboxArea = box(*areaInfo['bounds'])
    t=str(areaInfo['type'])

    #draw image png
    Img = rasterio.open(os.path.join(config.raw_image_base_dir,f'{config.raw_image_prefix}_{grid_id}{config.image_type}'))  
    sm_img = rasterio.mask.mask(Img, [bboxArea], all_touched=True, crop=True )
    profile_img = Img.profile  
    profile_img['height'] = sm_img[0].shape[1]
    profile_img['width'] = sm_img[0].shape[2]
    profile_img['transform'] = sm_img[1]
        # That's a problem with rasterio, if the height and the width are less then 256 it throws: ValueError: blockysize exceeds raster height 
        # So set the blockxsize and blockysize to prevent this problem
    profile_img['blockxsize'] = 32
    profile_img['blockysize'] = 32
    profile_img['dtype'] = rasterio.float32    
    profile_img['compress'] = 'lzw'
    profile_img['tiled'] = 'True'
    profile_img['nodata'] =None
    print(profile_img) 
    # To save storage space, image values were retained to three decimal places and multiplied by 1000 before being stored in integer format.
    # Here, the values are converted back to floating point.
    
    dt_img = sm_img[0].astype(profile_img['dtype'])
    with rasterio.open(os.path.join(writePath, f'{config.extracted_image_filename}{t}_{file_name}{config.image_type}'), 'w', **profile_img) as dst:
        dst.write(dt_img) 

#     draw annotation png
    polygons = []
    for i in polygonsInAreaDf.index:
        gm = polygonsInAreaDf.loc[i]['geometry']
        polygons.append(gm)
    profile_img['count'] = 1    
    with rasterio.open(os.path.join(writePath,f'{config.extracted_annotation_filename}{t}_{file_name}{config.ann_type}'), 'w+', **profile_img) as out:
        out_arr = out.read(1)
        burned = features.rasterize(polygons, fill=0, default_value=1,out=out_arr, transform=out.transform)
        out.write_band(1, burned)


In [ ]:
for key,value in zip(areasWithPolygons.keys(),areasWithPolygons.values()):
    dataset_dir=os.path.join(config.dataset_dir,r'{}'.format(value['dataset']))
    extractAreasThatOverlapWithTrainingData(value,dataset_dir)

In [ ]:
# Display extracted image 
file_name = ''
dataset_dir=os.path.join(config.dataset_dir,r'test' )
img_fn = os.path.join(dataset_dir, f'{config.extracted_image_filename}{file_name}{config.image_type}')
img = rasterio.open(img_fn)
read_img = img.read()

ann_fn=os.path.join(dataset_dir, f'{config.extracted_annotation_filename}{file_name}{config.ann_type}')
ann=rasterio.open(ann_fn)
read_annotation = ann.read()
comb_img=np.concatenate((read_img,read_annotation), axis=0)
comb_img=np.transpose(comb_img, axes=(1,2,0))
# print(read_annotation.shape)
# print(read_annotation)

print(comb_img.shape)
display_images(np.expand_dims(comb_img, axis=0),titles=['ndwi','rgb','swir','annotation'])